# Test field for generic processing functions

## 1. Packages

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import os
import cv2
from scipy import fftpack
import time
from nd2reader import ND2Reader

## 2. Bpass

Combining high pass and low pass filters in one function. This was originally done by directly Fourier Transform of the original image and then set 0 the undesired frequencies. Finally, the process Fourier (frequency) domain will be converted back to space domain.

I remember that if the images have shape 2$^n$, the FT can be computed more efficiently. Let's do the test. 

### 2.1 Compare FT efficiency between original image and padded image

- original image shape: (682, 682)
- padded image shape: (1024, 1024)

In [19]:
folder = r'E:\moreData\02042020\20-1\8-bit'
img = io.imread(os.path.join(folder, '0000.tif'))
dsize = (img.shape[0]//3, img.shape[1]//3)
img_s = cv2.resize(img, dsize)
dsize

(682, 682)

In [30]:
target_size = int(2 ** (np.log(max(dsize))//np.log(2) + 1))
print('target size: ' + str(target_size))

target size: 1024


In [31]:
vert_pad_before = int((target_size - dsize[0]) // 2)
vert_pad_after = target_size - dsize[0] - vert_pad_before
horiz_pad_before = int((target_size - dsize[1]) // 2)
horiz_pad_after = target_size - dsize[1] - horiz_pad_before
pad_width = ((vert_pad_before, vert_pad_after), (horiz_pad_before, horiz_pad_after))
print('pad_width: ' + str(pad_width))

pad_width: ((171, 171), (171, 171))


In [32]:
padded = np.pad(img_s, pad_width)
padded.shape

(1024, 1024)

In [61]:
tic = time.monotonic()
im_fft_1 = fftpack.fft2(img_s)
toc = time.monotonic()
t1 = toc - tic

tic = time.monotonic()
im_fft_2 = fftpack.fft2(padded)
toc = time.monotonic()
t2 = toc - tic

tic = time.monotonic()
im_fft_3 = np.fft.fft2(img_s)
toc = time.monotonic()
t3 = toc - tic

tic = time.monotonic()
im_fft_4 = np.fft.fft2(padded)
toc = time.monotonic()
t4 = toc - tic

print('fftpack.fft2 original image: ' + '{:.2f} ms'.format(t1*1000))
print('fftpack.fft2 padded image: ' + '{:.2f} ms'.format(t2*1000))
print('np.fft.fft2 original image: ' + '{:.2f} ms'.format(t3*1000))
print('np.fft.fft2 padded image: ' +'{:.2f} ms'.format(t4*1000))

fftpack.fft2 original image: 15.00 ms
fftpack.fft2 padded image: 32.00 ms
np.fft.fft2 original image: 31.00 ms
np.fft.fft2 padded image: 31.00 ms


**Conclusion:** fftpack.fft2 has the best performance on the original image. No need to pad.

## 3. Retrieve metadata from .nd2 files

Although so far all the .nd2 files I have seen encodes the images in 11-bit, there is a possible occasion that the bit depth got changed and the default converter to_tif.py may run into problem. If we can retrieve the bit depth of an .nd2 image before converting to 8-bit .tif, the converter would be much more robust! [ND2Reader](https://rbnvrw.github.io/nd2reader/nd2reader.html) provides this function.

In [6]:
with ND2Reader(r'E:\Zhengyang\08032020\00.nd2') as images:
    meta = images.metadata
a = ''
for kw in meta:
    a += kw + ', '

In [7]:
a

'height, width, date, fields_of_view, frames, z_levels, z_coordinates, total_images_per_channel, channels, pixel_microns, num_frames, experiment, events, '

ND2Reader does provide some metadata, namely (height, width, date, fields_of_view, frames, z_levels, z_coordinates, total_images_per_channel, channels, pixel_microns, num_frames, experiment, events). Unfortunately, it does not provide bit depth. So for now, we still use the old convention by defaulting the bit depth to 11.